In [ ]:
import os
import sys
import time

import pandas

In [8]:
if os.getenv("COLAB_RELEASE_TAG"):
   !pip install phitter --quiet
else:
   sys.path.append("../../../")

In [9]:
import phitter

In [10]:
df_fit_time = pandas.DataFrame()

In [11]:
IDS_DISTRIBUTION_FIT = ["beta", "fatigue_life", "triangular", "exponential_2p", "power_function", "normal", "f_4p", "johnson_sb"]
SAMPLE_SIZES = [1000, 10000, 100000, 500000, 1000000]
WORKERS = [1, 2, 6, 10, 20]

In [ ]:
for sample_size in SAMPLE_SIZES:
    for n_workers in WORKERS:
        for id_distribution in IDS_DISTRIBUTION_FIT:
            distribution_class = phitter.continuous.CONTINUOUS_DISTRIBUTIONS[id_distribution]
            data = distribution_class(init_parameters_examples=True).sample(sample_size)
            ti = time.time()
            phi = phitter.Phitter(data=data)
            phi.fit(n_workers=n_workers)
            tf = time.time() - ti
            df_fit_time = df_fit_time.fillna(0)
            if n_workers in df_fit_time.columns and sample_size in df_fit_time.index:
                df_fit_time.loc[sample_size, n_workers] += tf
            else:
                df_fit_time.loc[sample_size, n_workers] = tf

In [13]:
df_fit_time

,1,2,6,10,20
1000,66.385011,56.993431,71.733937,79.429709,129.796835
10000,166.968503,114.117879,84.489277,92.802805,142.849921
100000,1221.036816,777.887108,461.847912,412.945416,425.850307
500000,7319.432771,5126.522601,2960.258715,2139.677649,2062.027165
1000000,12646.801178,7779.188330,4588.343440,3972.455426,3406.246946


In [14]:
df_fit_time = df_fit_time / len(IDS_DISTRIBUTION_FIT)
df_fit_time

,1,2,6,10,20
1000,8.298126,7.124179,8.966742,9.928714,16.224604
10000,20.871063,14.264735,10.561160,11.600351,17.856240
100000,152.629602,97.235889,57.730989,51.618177,53.231288
500000,914.929096,640.815325,370.032339,267.459706,257.753396
1000000,1580.850147,972.398541,573.542930,496.556928,425.780868


In [15]:
print(df_fit_time.to_markdown(floatfmt=".4f"))

|         |         1 |        2 |        6 |       10 |       20 |
|--------:|----------:|---------:|---------:|---------:|---------:|
|    1000 |    8.2981 |   7.1242 |   8.9667 |   9.9287 |  16.2246 |
|   10000 |   20.8711 |  14.2647 |  10.5612 |  11.6004 |  17.8562 |
|  100000 |  152.6296 |  97.2359 |  57.7310 |  51.6182 |  53.2313 |
|  500000 |  914.9291 | 640.8153 | 370.0323 | 267.4597 | 257.7534 |
| 1000000 | 1580.8501 | 972.3985 | 573.5429 | 496.5569 | 425.7809 |
